In [1]:
using Plots;
gadfly();

In [2]:
include("../fdtd/update.jl");
include("../fdtd/sources.jl");
include("../fdtd/boundaries.jl");
using update;
using sources;

In [31]:
#Global parameters
size = 200;
endTime = 1200;
num_snaps = 300;
snap_step = div(endTime, num_snaps);

n1 = 4
n2 = sqrt(n1)

eps1 = n1^2;
eps2 = n2^2;

wavelength = 80
#Grid

# Magnetic
hy = zeros(size-1);
mu = ones(size-1);

chyh = ones(size);
chye = ones(size);


# Electric
ez = zeros(size);
eps = ones(size) * eps1;

cezh = ones(size);
ceze = ones(size);


#for i in 110:170
#    eps[i] = eps1;
#end
for i in 100:100+(div(wavelength, 2))
    eps[i] = eps2;
end

boundaries.setup_first_order_abc!(eps, mu, size, true)
boundaries.setup_first_order_abc!(eps, mu, 1, false)

# output params
ez_snapshot = Array{Any}(num_snaps);
hy_snapshot = Array{Any}(num_snaps);

In [32]:
# Time steps

for time in 1:endTime
    # Incident
    delay = 30.
    width = 100.
 
    ez_inc = sin(2*pi/wavelength * time);
    hy_inc = sin(2*pi/wavelength * (time - 0.5 - 0.5));
    
    #
    # Magnetic
    #
        
    # Interior update
    update.update_magnetic_field!(ez, hy, mu, chyh, chye);    
    
    # TFSF
    hy[49] -= hy_inc / globals.imp0;
    
    #
    # Electric
    #
       
    # Interior update
    update.update_electric_field!(ez, hy, eps, cezh, ceze);
    
    # ABC
    boundaries.first_order_diff_abc!(ez, 1, false)
    boundaries.first_order_diff_abc!(ez, size, true)
    
    # TFSF
    ez[50] += ez_inc / sqrt( eps[50] * mu[50])
 
    # Snapshots for animation
    if mod(time, snap_step) == 0
        ez_snapshot[div(time,snap_step)] = (time, copy(ez))
        hy_snapshot[div(time,snap_step)] = (time, copy(hy).*globals.imp0)        
    end
    
end

In [35]:
anim = Animation()

for i = 1:num_snaps
    p = plot(1:size, ez_snapshot[i][2], lab="Ez")
    # plot!(1:size-1, hy_snapshot[i][2], lab="Hy*imp0")
    
    time = ez_snapshot[i][1]
    plot!(ann=[(150, 1.5, "time =$time")])
    plot!(ann=[(0, 1.1, "Mur ABC")])
    
    plot!(ann=[(75, 1.3, "Eps = $eps1")])
    plot!(ann=[(102, 1.1, "Eps =\n= $eps2")])
    plot!(ann=[(125, 1.3, "Eps = $eps1")])
    plot!([100, 100], [-2, 2])
    plot!([120, 120], [-2, 2])
    
    plot!(ann=[(175, 1.1, "Mur ABC")])
    
    plot!(xlims=(1, 200), ylims=(-2, 2))
    frame(anim, p)
end
gif(anim, "./11_harmonic.gif", fps=15)

INFO: Saved animation to /media/storage/Documents/Github/1d-fdtd/examples/11_harmonic.gif


Plots.AnimatedGif("/media/storage/Documents/Github/1d-fdtd/examples/11_harmonic.gif")